In [ ]:
import os, json
import requests
import pandas as pd
import time, datetime
import pprint as pp
from dotenv import load_dotenv
load_dotenv()

In [ ]:
BASE_URL = os.getenv("MYSHYN_URL")

TENANT_ID = os.getenv("MYSHYN_TENANT_ID")

USERNAME = os.getenv("MYSHYN_USERNAME")
PASSWORD = os.getenv("MYSHYN_PASSWORD")

PROXIES = {
    "https" : os.getenv("PROXIES_HTTPS")
}

In [ ]:
HEADER = {
    'Content-Type': 'application/json'
}

In [ ]:
def generateDateAndLast7day(date: None):
    # Get date
    selectedTime = datetime.datetime.strptime(date, '%d/%m/%y')
    last7dayTime = selectedTime - datetime.timedelta(days=6)
    
    # Convert date to timestamp
    selectedTime = selectedTime.strftime('%Y-%m-%dT23:59:00')
    last7dayTime = last7dayTime.strftime('%Y-%m-%dT00:00:00')

    # Convert timestamp to date
    # nowTime = datetime.datetime.fromtimestamp(nowTime)
    # last7dayTime = datetime.datetime.fromtimestamp(last7dayTime)
    
    return last7dayTime, selectedTime

In [ ]:
last7dayTime, selectedTime = generateDateAndLast7day(date='21/02/23')
last7dayTime, selectedTime

## 1. Query User Acitivity

```
maximum log : 500
```

In [ ]:
def getUserActivity(startTime="", serviceNames=[]):
    url = BASE_URL + '/shnapi/rest/external/api/v1/queryIncidents?limit=500'
    print("URL  : " + url)

    BODY =  { 
        "startTime": startTime,
        "serviceNames": serviceNames,
#         "incidentCriteria": { 
#             "categories": [ 
#                 {
#                     "incidentType":"Threat"
#                 }
#             ] 
#         } 
    } 
    
    allresult = []

    while True:
        r = requests.post(url, headers=HEADER, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), proxies=PROXIES, verify=False)
        print(r.status_code)

        res = r.json()["body"]

        allresult += res["incidents"]
        
        if res["responseInfo"]["nextStartTime"] != None and res["incidents"] != []:
            BODY["startTime"] = res["responseInfo"]["nextStartTime"]
            print(BODY["startTime"])
        else:
            print(res["responseInfo"]["nextStartTime"])
            return allresult

In [ ]:
startTime = selectedTime
serviceNames = ["sharepoint"] # ,"onedrive","exchange"

In [ ]:
allUserAcitivities = getUserActivity(startTime=startTime, serviceNames=serviceNames)

In [ ]:
allUserAcitivities_df = pd.DataFrame(allUserAcitivities)
allUserAcitivities_df

## 2. Query Incident Information Keys

In [ ]:
url = BASE_URL + '/shnapi/rest/external/api/v1/queryIncidentInformationKeys'
print("URL  : " + url)

r = requests.post(url, headers=HEADER, params={}, data={}, auth=(USERNAME,PASSWORD), proxies=PROXIES, verify=False)
r.status_code, r.text

In [ ]:
r.json()